In [12]:
import numpy as np
import pandas as pd

In [13]:
data = pd.read_csv('flights.csv', parse_dates=["FL_DATE"]).rename(columns=str.lower)

In [14]:
data.head()

,year,quarter,month,day_of_month,day_of_week,fl_date,unique_carrier,airline_id,carrier,tail_num,...,dest_state_nm,dep_time,dep_delay,dep_delay_new,dep_del15,taxi_out,wheels_off,wheels_on,taxi_in,unnamed: 33
0,2015,1,1,1,4,2015-01-01,AA,19805,AA,N787AA,...,California,855.0,-5.0,0.0,0.0,17.0,912.0,1230.0,7.0,NaN
1,2015,1,1,2,5,2015-01-02,AA,19805,AA,N795AA,...,California,850.0,-10.0,0.0,0.0,15.0,905.0,1202.0,9.0,NaN
2,2015,1,1,3,6,2015-01-03,AA,19805,AA,N788AA,...,California,853.0,-7.0,0.0,0.0,15.0,908.0,1138.0,13.0,NaN
3,2015,1,1,4,7,2015-01-04,AA,19805,AA,N791AA,...,California,853.0,-7.0,0.0,0.0,14.0,907.0,1159.0,19.0,NaN
4,2015,1,1,5,1,2015-01-05,AA,19805,AA,N783AA,...,California,853.0,-7.0,0.0,0.0,27.0,920.0,1158.0,24.0,NaN


### User defined functions: get_regions, get_weekday, and get_carrier_name

In [15]:
def get_regions(df,column):
    
    travel_regions = {
     'Alabama':'South','Alaska':'West','Arizona':'SouthWest','Arkansas':'South',
     'California':'West','Colorado':'West','Connecticut':'New England',
     'Delaware':'Mid-Atlantic','Florida':'South','Georgia':'South','Hawaii':'West',
     'Idaho':'West','Illinois':'MidWest','Indiana':'MidWest','Iowa':'MidWest',
     'Kansas':'MidWest','Kentucky':'South','Louisiana':'South','Maine':'New England',
     'Maryland':'Mid-Atlantic','Massachusetts':'New England','Michigan':'MidWest',
     'Minnesota':'MidWest','Mississippi':'South','Missouri':'MidWest','Montana':'West',
     'Nebraska':'MidWest','Nevada':'West','New Hampshire':'New England',
     'New Jersey':'Mid-Atlantic','New Mexico':'SouthWest','New York':'Mid-Atlantic',
     'North Carolina':'South','North Dakota':'MidWest','Ohio':'MidWest',
     'Oklahoma':'SouthWest','Oregon':'West','Pennsylvania':'Mid-Atlantic',
     'Puerto Rico':'Puerto Rico','Rhode Island':'New England','South Carolina':'South',
     'South Dakota':'MidWest','Tennessee':'South','Texas':'SouthWest',
     'U.S. Virgin Islands':'U.S. Virgin Islands','Utah':'West','Vermont':'New England',
     'Virginia':'South','Washington D.C.':'Mid-Atlantic','Washington':'West',
     'West Virginia':'South','Wisconsin':'MidWest','Wyoming':'West',
     'U.S. Pacific Trust Territories and Possessions':'U.S. Pacific Trust Territories and Possessions'}
    
    df[column.name+'_region'] = [travel_regions[c] for c in column]
    
    return df

In [16]:
def get_weekday(df,column):
    
    day_of_week = {1:'Monday',
                   2:'Tuesday',
                   3:'Wednesday',
                   4:'Thursday',
                   5:'Friday',
                   6:'Saturday',
                   7:'Sunday'}
    df['weekday'] = [day_of_week[c] for c in column]
    return df

In [17]:
def get_carrier_name(df,column):
    carriers = {
            'WN':'Southwest Airlines',
            'HA':'Hawaiian Airlines',
            'DL':'Delta Air Lines',
            'AA':'American Airlines',
            'AS':'Alaska Airlines',
            'UA':'United Air Lines',
            'OO':'SkyWest Airlines',
            'US' :'US Airways',
            'EV':'ExpressJet Airlines',
            'MQ':'Envoy Air',
            'B6':'JetBlue Airways',
            'NK':'Spirit Air Lines',
            'F9':'Frontier Airlines',
            'VX':'Virgin America'}
    
    df[column.name+'_long_name'] = [carriers[c] for c in column]
    return df

In [18]:
results_data = (data
              .pipe(get_regions, data.dest_state_nm)   # Apply UDF based on dest_state_nam 

              .pipe(get_regions, data.origin_state_nm) # Apply UDF based on origin_state_nm 
              
              .pipe(get_carrier_name, data.carrier)    # Apply UDF based on carrier_name          
              
              .pipe(get_weekday, data.day_of_week)     # Apply UDF based on day_of_week
              
              .drop('unnamed: 33', axis=1)             # Drop column
              
              .assign(hour_of_day = data.dep_time      # Add new column
                      .fillna(0)
                      .apply(lambda x: int(str(x/100).split('.')[0]))) 
             )

In [28]:
columns = ['carrier_long_name','dest_state_nm_region', 
           'origin_state_nm_region','weekday',
           'hour_of_day','tail_num']

report_df = results_data[columns].groupby(columns[:5]).count().copy()

In [26]:
report_df.head(25)

tail_num
carrier_long_name dest_state_nm_region origin_state_nm_region weekday  hour_of_day          
Alaska Airlines   Mid-Atlantic         West                   Friday   8                  14
                                                                       9                   1
                                                                       15                  1
                                                                       16                  4
                                                              Monday   0                   1
                                                                       8                  10
                                                                       9                   1
                                                                       10                  1
                                                                       15                  2
                                                                       16                  1
                                                              Saturday 8                  13
                                                                       9                   2
                                                                       15                  1
                                                              Sunday   8                   9
                                                                       9                   2
                                                                       10                  1
                                                                       15                  1
                                                                       16                  2
                                                                       17                  1
                                                              Thursday 8                  13
                                                                       9                   1
                                                                       10                  1
                                                                       15                  3
                                                                       16                  1
                                                                       17                  1